In [2]:
from __future__ import print_function
import numpy as np

In [15]:
class RBM:
  
    def __init__(self, num_visible, num_hidden):
        self.num_hidden = num_hidden
        self.num_visible = num_visible
        self.debug_print = True

        # Initialize a weight matrix, of dimensions (num_visible x num_hidden), using
        # a uniform distribution between -sqrt(6. / (num_hidden + num_visible))
        # and sqrt(6. / (num_hidden + num_visible)). One could vary the 
        # standard deviation by multiplying the interval with appropriate value.
        # Here we initialize the weights with mean 0 and standard deviation 0.1. 
        # Reference: Understanding the difficulty of training deep feedforward 
        # neural networks by Xavier Glorot and Yoshua Bengio
        np_rng = np.random.RandomState(1234)

        self.weights = np.asarray(np_rng.uniform(
                low=-0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                            high=0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                            size=(num_visible, num_hidden)))


        # Insert weights for the bias units into the first row and first column.
        self.weights = np.insert(self.weights, 0, 0, axis = 0)
        self.weights = np.insert(self.weights, 0, 0, axis = 1)

    def train(self, data, max_epochs = 1000, learning_rate = 0.1):
        """
        Train the machine.

        Parameters
        ----------
        data: A matrix where each row is a training example consisting of the states of visible units.    
        """

        num_examples = data.shape[0]

        # Insert bias units of 1 into the first column.
        data = np.insert(data, 0, 1, axis = 1)

        for epoch in range(max_epochs):      
            # Clamp to the data and sample from the hidden units. 
            # (This is the "positive CD phase", aka the reality phase.)
            pos_hidden_activations = np.dot(data, self.weights)      
            pos_hidden_probs = self._logistic(pos_hidden_activations)
            pos_hidden_probs[:,0] = 1 # Fix the bias unit.
            pos_hidden_states = pos_hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
            # Note that we're using the activation *probabilities* of the hidden states, not the hidden states       
            # themselves, when computing associations. We could also use the states; see section 3 of Hinton's 
            # "A Practical Guide to Training Restricted Boltzmann Machines" for more.
            pos_associations = np.dot(data.T, pos_hidden_probs)

            # Reconstruct the visible units and sample again from the hidden units.
            # (This is the "negative CD phase", aka the daydreaming phase.)
            neg_visible_activations = np.dot(pos_hidden_probs, self.weights.T)
            neg_visible_probs = self._logistic(neg_visible_activations)
            neg_visible_probs[:,0] = 1 # Fix the bias unit.
            neg_hidden_activations = np.dot(neg_visible_probs, self.weights)
            neg_hidden_probs = self._logistic(neg_hidden_activations)
            # Note, again, that we're using the activation *probabilities* when computing associations, not the states 
            # themselves.
            neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)

            # Update weights.
            self.weights += learning_rate * ((pos_associations - neg_associations) / num_examples)

            error = np.sum((data - neg_visible_probs) ** 2)
            if self.debug_print:
                print("Epoch %s: error is %s" % (epoch, error))

    def run_visible(self, data):
        """
        Assuming the RBM has been trained (so that weights for the network have been learned),
        run the network on a set of visible units, to get a sample of the hidden units.

        Parameters
        ----------
        data: A matrix where each row consists of the states of the visible units.

        Returns
        -------
        hidden_states: A matrix where each row consists of the hidden units activated from the visible
        units in the data matrix passed in.
        """

        num_examples = data.shape[0]

        # Create a matrix, where each row is to be the hidden units (plus a bias unit)
        # sampled from a training example.
        hidden_states = np.ones((num_examples, self.num_hidden + 1))

        # Insert bias units of 1 into the first column of data.
        data = np.insert(data, 0, 1, axis = 1)

        # Calculate the activations of the hidden units.
        hidden_activations = np.dot(data, self.weights)
        # Calculate the probabilities of turning the hidden units on.
        hidden_probs = self._logistic(hidden_activations)
        # Turn the hidden units on with their specified probabilities.
        hidden_states[:,:] = hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
        # Always fix the bias unit to 1.
        # hidden_states[:,0] = 1

        # Ignore the bias units.
        hidden_states = hidden_states[:,1:]
        return hidden_states
    
        # TODO: Remove the code duplication between this method and `run_visible`?
    def run_hidden(self, data):
        """
        Assuming the RBM has been trained (so that weights for the network have been learned),
        run the network on a set of hidden units, to get a sample of the visible units.

        Parameters
        ----------
        data: A matrix where each row consists of the states of the hidden units.

        Returns
        -------
        visible_states: A matrix where each row consists of the visible units activated from the hidden
        units in the data matrix passed in.
        """

        num_examples = data.shape[0]

        # Create a matrix, where each row is to be the visible units (plus a bias unit)
        # sampled from a training example.
        visible_states = np.ones((num_examples, self.num_visible + 1))

        # Insert bias units of 1 into the first column of data.
        data = np.insert(data, 0, 1, axis = 1)

        # Calculate the activations of the visible units.
        visible_activations = np.dot(data, self.weights.T)
        # Calculate the probabilities of turning the visible units on.
        visible_probs = self._logistic(visible_activations)
        # Turn the visible units on with their specified probabilities.
        visible_states[:,:] = visible_probs > np.random.rand(num_examples, self.num_visible + 1)
        # Always fix the bias unit to 1.
        # visible_states[:,0] = 1

        # Ignore the bias units.
        visible_states = visible_states[:,1:]
        return visible_states
    
    def daydream(self, num_samples):
        """
        Randomly initialize the visible units once, and start running alternating Gibbs sampling steps
        (where each step consists of updating all the hidden units, and then updating all of the visible units),
        taking a sample of the visible units at each step.
        Note that we only initialize the network *once*, so these samples are correlated.

        Returns
        -------
        samples: A matrix, where each row is a sample of the visible units produced while the network was
        daydreaming.
        """

        # Create a matrix, where each row is to be a sample of of the visible units 
        # (with an extra bias unit), initialized to all ones.
        samples = np.ones((num_samples, self.num_visible + 1))

        # Take the first sample from a uniform distribution.
        samples[0,1:] = np.random.rand(self.num_visible)

        # Start the alternating Gibbs sampling.
        # Note that we keep the hidden units binary states, but leave the
        # visible units as real probabilities. See section 3 of Hinton's
        # "A Practical Guide to Training Restricted Boltzmann Machines"
        # for more on why.
        for i in range(1, num_samples):
            visible = samples[i-1,:]

            # Calculate the activations of the hidden units.
            hidden_activations = np.dot(visible, self.weights)      
            # Calculate the probabilities of turning the hidden units on.
            hidden_probs = self._logistic(hidden_activations)
            # Turn the hidden units on with their specified probabilities.
            hidden_states = hidden_probs > np.random.rand(self.num_hidden + 1)
            # Always fix the bias unit to 1.
            hidden_states[0] = 1

            # Recalculate the probabilities that the visible units are on.
            visible_activations = np.dot(hidden_states, self.weights.T)
            visible_probs = self._logistic(visible_activations)
            visible_states = visible_probs > np.random.rand(self.num_visible + 1)
            samples[i,:] = visible_states

        # Ignore the bias units (the first column), since they're always set to 1.
        return samples[:,1:]        
      
    def _logistic(self, x):
        return 1.0 / (1 + np.exp(-x))

In [19]:
if __name__ == '__main__':
    r = RBM(num_visible = 6, num_hidden = 2)
    training_data = np.array([[1,1,1,0,0,0],[1,0,1,0,0,0],[1,0,1,0,0,0],[0,0,1,1,1,0], [0,0,1,1,0,0],[0,0,1,1,1,0]])
    r.train(training_data, max_epochs = 5000)
    print(r.weights)
    user = np.array([[0,0,0,1,1,0]])
    print(r.run_visible(user))

Epoch 0: error is 9.030333143940396
Epoch 1: error is 8.782243991596232
Epoch 2: error is 8.54772687558609
Epoch 3: error is 8.326779916982012
Epoch 4: error is 8.119200625267407
Epoch 5: error is 7.924627740438095
Epoch 6: error is 7.742582059939559
Epoch 7: error is 7.572503280046175
Epoch 8: error is 7.413781215408362
Epoch 9: error is 7.265780868616916
Epoch 10: error is 7.127861603633813
Epoch 11: error is 6.99939113513891
Epoch 12: error is 6.879755241585545
Epoch 13: error is 6.768364124391034
Epoch 14: error is 6.664656244926127
Epoch 15: error is 6.568100332937843
Epoch 16: error is 6.478196113081274
Epoch 17: error is 6.394474161897297
Epoch 18: error is 6.316495195363374
Epoch 19: error is 6.243848999054694
Epoch 20: error is 6.176153146941933
Epoch 21: error is 6.113051607181362
Epoch 22: error is 6.05421329986214
Epoch 23: error is 5.999330648882289
Epoch 24: error is 5.9481181549020885
Epoch 25: error is 5.900311006338091
Epoch 26: error is 5.855663738897558
Epoch 27: err

Epoch 829: error is 0.9449162609429266
Epoch 830: error is 0.9439865176076271
Epoch 831: error is 0.94305843324918
Epoch 832: error is 0.9421320258205226
Epoch 833: error is 0.9412073131666198
Epoch 834: error is 0.940284313018573
Epoch 835: error is 0.9393630429878693
Epoch 836: error is 0.9384435205607711
Epoch 837: error is 0.9375257630928504
Epoch 838: error is 0.9366097878036644
Epoch 839: error is 0.9356956117715761
Epoch 840: error is 0.9347832519287164
Epoch 841: error is 0.9338727250560963
Epoch 842: error is 0.9329640477788574
Epoch 843: error is 0.9320572365616746
Epoch 844: error is 0.9311523077042978
Epoch 845: error is 0.9302492773372438
Epoch 846: error is 0.9293481614176314
Epoch 847: error is 0.9284489757251633
Epoch 848: error is 0.9275517358582509
Epoch 849: error is 0.9266564572302857
Epoch 850: error is 0.9257631550660543
Epoch 851: error is 0.924871844398298
Epoch 852: error is 0.9239825400644133
Epoch 853: error is 0.9230952567032983
Epoch 854: error is 0.9222100

Epoch 1738: error is 0.6811296025480618
Epoch 1739: error is 0.6810784630592157
Epoch 1740: error is 0.6810274353686662
Epoch 1741: error is 0.6809765191101508
Epoch 1742: error is 0.6809257139188056
Epoch 1743: error is 0.6808750194311622
Epoch 1744: error is 0.6808244352851389
Epoch 1745: error is 0.6807739611200354
Epoch 1746: error is 0.6807235965765267
Epoch 1747: error is 0.680673341296656
Epoch 1748: error is 0.6806231949238295
Epoch 1749: error is 0.6805731571028089
Epoch 1750: error is 0.6805232274797065
Epoch 1751: error is 0.6804734057019788
Epoch 1752: error is 0.6804236914184194
Epoch 1753: error is 0.6803740842791545
Epoch 1754: error is 0.680324583935635
Epoch 1755: error is 0.6802751900406325
Epoch 1756: error is 0.6802259022482318
Epoch 1757: error is 0.6801767202138251
Epoch 1758: error is 0.6801276435941076
Epoch 1759: error is 0.6800786720470693
Epoch 1760: error is 0.6800298052319907
Epoch 1761: error is 0.6799810428094367
Epoch 1762: error is 0.6799323844412503
Ep

Epoch 2709: error is 0.6552364721303219
Epoch 2710: error is 0.6552184246039101
Epoch 2711: error is 0.6552003753428818
Epoch 2712: error is 0.6551823243009921
Epoch 2713: error is 0.6551642714319782
Epoch 2714: error is 0.6551462166895606
Epoch 2715: error is 0.6551281600274407
Epoch 2716: error is 0.6551101013993006
Epoch 2717: error is 0.6550920407588046
Epoch 2718: error is 0.6550739780595973
Epoch 2719: error is 0.6550559132553034
Epoch 2720: error is 0.6550378462995283
Epoch 2721: error is 0.6550197771458556
Epoch 2722: error is 0.6550017057478503
Epoch 2723: error is 0.654983632059055
Epoch 2724: error is 0.6549655560329912
Epoch 2725: error is 0.65494747762316
Epoch 2726: error is 0.6549293967830382
Epoch 2727: error is 0.6549113134660822
Epoch 2728: error is 0.6548932276257248
Epoch 2729: error is 0.6548751392153764
Epoch 2730: error is 0.6548570481884239
Epoch 2731: error is 0.6548389544982299
Epoch 2732: error is 0.6548208580981341
Epoch 2733: error is 0.6548027589414507
Epo

Epoch 3757: error is 0.6186114196302169
Epoch 3758: error is 0.6185184713778321
Epoch 3759: error is 0.6184251947369162
Epoch 3760: error is 0.6183315879648916
Epoch 3761: error is 0.6182376493074359
Epoch 3762: error is 0.618143376998389
Epoch 3763: error is 0.6180487692596537
Epoch 3764: error is 0.6179538243011014
Epoch 3765: error is 0.6178585403204739
Epoch 3766: error is 0.617762915503283
Epoch 3767: error is 0.6176669480227126
Epoch 3768: error is 0.617570636039517
Epoch 3769: error is 0.6174739777019205
Epoch 3770: error is 0.6173769711455137
Epoch 3771: error is 0.6172796144931504
Epoch 3772: error is 0.6171819058548436
Epoch 3773: error is 0.6170838433276599
Epoch 3774: error is 0.6169854249956129
Epoch 3775: error is 0.6168866489295566
Epoch 3776: error is 0.6167875131870758
Epoch 3777: error is 0.6166880158123776
Epoch 3778: error is 0.6165881548361813
Epoch 3779: error is 0.6164879282756058
Epoch 3780: error is 0.6163873341340581
Epoch 3781: error is 0.6162863704011197
Epo

Epoch 4814: error is 0.16825664442051347
Epoch 4815: error is 0.16804893882270366
Epoch 4816: error is 0.1678416324312092
Epoch 4817: error is 0.16763472434811727
Epoch 4818: error is 0.16742821367775337
Epoch 4819: error is 0.16722209952667108
Epoch 4820: error is 0.16701638100364402
Epoch 4821: error is 0.1668110572196558
Epoch 4822: error is 0.16660612728789131
Epoch 4823: error is 0.16640159032372837
Epoch 4824: error is 0.16619744544472806
Epoch 4825: error is 0.16599369177062664
Epoch 4826: error is 0.16579032842332703
Epoch 4827: error is 0.16558735452688964
Epoch 4828: error is 0.1653847692075248
Epoch 4829: error is 0.1651825715935834
Epoch 4830: error is 0.1649807608155498
Epoch 4831: error is 0.16477933600603292
Epoch 4832: error is 0.16457829629975806
Epoch 4833: error is 0.16437764083355946
Epoch 4834: error is 0.16417736874637204
Epoch 4835: error is 0.16397747917922345
Epoch 4836: error is 0.1637779712752267
Epoch 4837: error is 0.16357884417957216
Epoch 4838: error is 0